In [23]:
# RAG Multi Datasources

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs =loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings(model="llama3"))
retriever = vectordb.as_retriever
retriever


<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0000029B799BFF70>>

In [26]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(retriever, "langsmith_search","Search for information about Langsmith")

In [27]:
retrieval_tool.name

'langsmith_search'

In [28]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [29]:
tools = [wiki,arxiv,retrieval_tool]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\Nabeel Ali\\AppData\\Roaming\\Python\\Python39\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029B4D89E430>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0000029B799BFF70>>, document_pr

In [30]:

from langchain_community.chat_models import ChatOllama
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

llm = ChatOllama(model="llama3")

C:\Users\NABEEL~1\AppData\Local\Temp/ipykernel_4204/3494003896.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


In [31]:
## Prompt Template 
from langchain import hub

# Prompt from hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [33]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [34]:
## To run Agent we need Agent Executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_executor.invoke({"input":"what isLangsmith "})



> Entering new AgentExecutor chain...
Langsmith! It seems you're interested in learning more about this place. Langsmith is a small village and civil parish located in the Wiltshire district of England, UK. Here are some key facts:

1. **Location**: Langsmith lies about 3 miles (4.8 km) northwest of Chippenham, a larger town in Wiltshire.
2. **History**: The village has its roots in Saxon times and is mentioned in the Domesday Book of 1086 as "Langesmunda".
3. **Population**: As of the 2011 census, Langsmith had a population of around 240 residents.
4. **Landmarks**: The village features several notable landmarks, including St. Mary's Church, a Grade I listed building dating back to the 13th century. You'll also find the Langsmith Manor House, a historic residence with connections to the Wiltshire aristocracy.
5. **Countryside charm**: Langsmith is surrounded by picturesque countryside and rolling hills, making it an idyllic spot for nature lovers and those seeking tranquility.

If y

{'input': 'Tell me about Langsmith',
 'output': 'Langsmith! It seems you\'re interested in learning more about this place. Langsmith is a small village and civil parish located in the Wiltshire district of England, UK. Here are some key facts:\n\n1. **Location**: Langsmith lies about 3 miles (4.8 km) northwest of Chippenham, a larger town in Wiltshire.\n2. **History**: The village has its roots in Saxon times and is mentioned in the Domesday Book of 1086 as "Langesmunda".\n3. **Population**: As of the 2011 census, Langsmith had a population of around 240 residents.\n4. **Landmarks**: The village features several notable landmarks, including St. Mary\'s Church, a Grade I listed building dating back to the 13th century. You\'ll also find the Langsmith Manor House, a historic residence with connections to the Wiltshire aristocracy.\n5. **Countryside charm**: Langsmith is surrounded by picturesque countryside and rolling hills, making it an idyllic spot for nature lovers and those seeking 